In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf 
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import  Counter
import nltk

import re

from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from scipy import stats
from scipy.stats import norm, skew 
from scipy.special import boxcox1p


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
data

In [ ]:
# Check if there are null values
data.isna().sum().sort_values(ascending = False)

In [ ]:
#  Dropping the column ID
x = data.drop(['target','id'],axis=1)
y = data['target']

In [ ]:
#  Checking the value count of classes
a = data['target'].value_counts()
a

In [ ]:
#  Label encoding the data
le = LabelEncoder()
y = le.fit_transform(data['target'])
y

In [ ]:
#  Splitting the dataset into train and validation
label_X_train,label_X_valid,y_train,y_valid =  train_test_split(x,y, test_size=0.3,
                                                      random_state=0)

In [ ]:
#  Trying different decision methods
# Gaussian model
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(label_X_train, y_train)
y_pred = gaussian.predict(label_X_valid)
acc_gaussian = round(accuracy_score(y_pred, y_valid) * 100, 2)
print(acc_gaussian)

In [ ]:
#  RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(label_X_train, y_train)
y_pred = randomforest.predict(label_X_valid)
acc_randomforest = round(accuracy_score(y_pred, y_valid) * 100, 2)
print(acc_randomforest)

In [ ]:
# XGBClassifier
from xgboost import XGBClassifier
model = XGBClassifier(random_state=1, n_estimators = 90, learning_rate = 0.01)
model.fit(label_X_train, y_train)
y_pred = model.predict(label_X_valid)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_valid,y_pred)*100)

In [ ]:
 #Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(label_X_train, y_train)
y_pred = gbk.predict(label_X_valid)
acc_gbk = round(accuracy_score(y_pred, y_valid) * 100, 2)
print(acc_gbk)

In [ ]:
#  to get the accuracy of the model at different parameters
from sklearn.metrics import accuracy_score

def get_accuracy(X_train,X_valid,y_train,y_valid, max_depth=5, n_estimators=100, max_leaf_nodes=None):
    model1 = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=1, 
                                )
    model1.fit(X_train,y_train)
    pred = model1.predict(X_valid)
    return mean_absolute_error(y_valid, pred)

In [ ]:
# depths = [5, 10, 50, 100, 500]
# best_accuracy, best_depth = -float('inf'), 0
# l=[]
# for d in depths:
#     acc = get_accuracy(label_X_train,label_X_valid,y_train,y_valid, d)
#     print(f"Depth:{d} \t Accuracy:{acc}")
#     l.append(acc)
#     if acc ==min(l):
#         best_accuracy, best_depth =acc,d
        
        
# print(f"Best accuracy is {best_accuracy} at depth {best_depth}")

In [ ]:
# num_leaf_nodes = [5, 10, 50, 100, 500]
# best_accuracy, best_max_leaf_nodes = -float('inf'), None
# l1=[]
# for leaf in num_leaf_nodes:
#     acc = get_accuracy(label_X_train,label_X_valid,y_train,y_valid, best_depth,leaf)
#     print(f"LeafNodes:{leaf} \t Accuracy:{acc}")
#     l1.append(acc)
#     if acc==min(l1):
#         best_accuracy,best_max_leaf_nodes = acc,leaf
        
# print(f"Best accuracy is {best_accuracy} at leaf node {best_max_leaf_nodes}")


In [ ]:
# n_estimator_trees = [100, 200, 300, 400, 500]
# best_accuracy, best_n_estimator = -float('inf'), 0
# l2=[]
# for n_trees in n_estimator_trees:
#     acc = get_accuracy(label_X_train, label_X_valid, y_train, y_valid, max_depth=best_depth, max_leaf_nodes=best_max_leaf_nodes,
#                       n_estimators=n_trees)
#     print(f"No. of trees: {n_trees} \t Accuracy: {acc}")
#     l2.append(acc)
#     if acc == min(l2):
#         best_accuracy, best_n_estimator = acc, n_trees
        
# print(f"Best accuracy is {best_accuracy} at {best_n_estimator} no. of trees")

In [ ]:
#  Using the above parameters to finalize the model and fit the dataset
# final_model = GradientBoostingClassifier(n_estimators=best_n_estimator, max_leaf_nodes=best_max_leaf_nodes,
#                                     max_depth=best_depth, random_state=1)

gbk.fit(label_X_train,y_train)


In [ ]:
test = test.drop('id',axis=1)
# preds = final_model.predict(test)
preds = gbk.predict_proba(test)

In [ ]:
preds

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
sub[['Class_1','Class_2', 'Class_3', 'Class_4','Class_5','Class_6', 'Class_7', 'Class_8', 'Class_9']] = preds
# submission_result = pd.DataFrame(preds,columns=['Class_1','Class_2','Class_3','Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])

In [ ]:
sub